# Comparativa de Operaciones Básicas: Pandas vs Polars vs Data.table

Este notebook compara el rendimiento de operaciones básicas entre las tres librerías principales de procesamiento de datos en Python.

In [ ]:
import pandas as pd
import polars as pl
import datatable as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import sys
sys.path.append('..')
from src.benchmarks import run_benchmark, create_sample_data

## 1. Configuración del Entorno de Pruebas

In [ ]:
# Crear datos de prueba
N_ROWS = 1_000_000
data = create_sample_data(N_ROWS)

# Verificar versiones de las librerías
print(f"Pandas version: {pd.__version__}")
print(f"Polars version: {pl.__version__}")
print(f"Datatable version: {dt.__version__}")

## 2. Pruebas de Rendimiento

In [ ]:
def benchmark_io_operations():
    results = {}
    
    # Pandas
    start = time()
    df_pd = pd.DataFrame(data)
    df_pd.to_csv('test_pandas.csv', index=False)
    df_read = pd.read_csv('test_pandas.csv')
    results['Pandas'] = time() - start
    
    # Polars
    start = time()
    df_pl = pl.DataFrame(data)
    df_pl.write_csv('test_polars.csv')
    df_read = pl.read_csv('test_polars.csv')
    results['Polars'] = time() - start
    
    # Data.table
    start = time()
    df_dt = dt.Frame(data)
    df_dt.to_csv('test_datatable.csv')
    df_read = dt.fread('test_datatable.csv')
    results['Data.table'] = time() - start
    
    return results

# Ejecutar benchmark
results = benchmark_io_operations()

# Visualizar resultados
plt.figure(figsize=(10, 6))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.title('Tiempo de Operaciones IO')
plt.ylabel('Tiempo (segundos)')
plt.show()